# \[FRAUD\] 데이터정리 시도(matrix로 lesson6따라하기 - 실패ㅎ)

김보람  
2023-08-22

# imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import torch

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics 

# embedding 
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder, AverageEmbedder, WeightedL1Embedder, WeightedL2Embedder

In [3]:
def build_graph_bipartite(df_input, graph_type=nx.Graph()):
    df=df_input.copy()
    mapping={x:node_id for node_id, x in enumerate(set(df["cc_num"].values.tolist()+\
                                                      df["merchant"].values.tolist()))}
    
    df["from"]=df["cc_num"].apply(lambda x:mapping[x])  #엣지의 출발점
    df["to"]=df["merchant"].apply(lambda x:mapping[x])  #엣지의 도착점
    
    df = df[['from', 'to', "amt", "is_fraud"]].groupby(['from','to']).agg({"is_fraud":"sum","amt":"sum"}).reset_index()
    df["is_fraud"]=df["is_fraud"].apply(lambda x:1 if x>0 else 0)
    
    G=nx.from_edgelist(df[["from","to"]].values, create_using=graph_type)
    
    nx.set_edge_attributes(G,{(int(x["from"]),int(x["to"])):x["is_fraud"] for idx, x in df[["from","to","is_fraud"]].iterrows()}, "label")  #엣지 속성 설정,각 속성의 사기 여부부     
    nx.set_edge_attributes(G,{(int(x["from"]),int(x["to"])):x["amt"] for idx,x in df[["from","to","amt"]].iterrows()}, "weight") # 엣지 속성 설정, 각 엣지의 거래 금액

    return G


def build_graph_tripartite(df_input, graph_type=nx.Graph()):
    df=df_input.copy()
    mapping={x:node_id for node_id, x in enumerate(set(df.index.values.tolist() + 
                                                       df["cc_num"].values.tolist() +
                                                       df["merchant"].values.tolist()))}
    df["in_node"]= df["cc_num"].apply(lambda x: mapping[x])
    df["out_node"]=df["merchant"].apply(lambda x:mapping[x])
    
        
    G=nx.from_edgelist([(x["in_node"], mapping[idx]) for idx, x in df.iterrows()] +\
                        [(x["out_node"], mapping[idx]) for idx, x in df.iterrows()], create_using=graph_type)
    
    nx.set_edge_attributes(G,{(x["in_node"], mapping[idx]):x["is_fraud"] for idx, x in df.iterrows()}, "label")     
    nx.set_edge_attributes(G,{(x["out_node"], mapping[idx]):x["is_fraud"] for idx, x in df.iterrows()}, "label")   
    nx.set_edge_attributes(G,{(x["in_node"], mapping[idx]):x["amt"] for idx, x in df.iterrows()}, "weight")  
    nx.set_edge_attributes(G,{(x["out_node"], mapping[idx]):x["amt"] for idx, x in df.iterrows()}, "weight")

    return G
    
    
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

def embedding(Graph):
    # Graph -> X (feature)
    _edgs = list(Graph.edges)
    subGraph = Graph.edge_subgraph([_edgs[x] for x in range(len(Graph.edges))]).copy()
    subGraph.add_nodes_from(list(set(Graph.nodes) - set(subGraph.nodes)))    
    embedded = AverageEmbedder(Node2Vec(subGraph, weight_key='weight').fit(window=10).wv)
    X = [embedded[str(_edgs[x][0]), str(_edgs[x][1])] for x in range(len(Graph.edges))]
    # Graph -> y (label)
    y = np.array(list(nx.get_edge_attributes(Graph, "label").values()))
    return X,y 

def anal(df):
    Graph = build_graph_bipartite(df)
    X,XX,y,yy = embedding(Graph)
    lrnr = RandomForestClassifier(n_estimators=100, random_state=42) 
    lrnr.fit(X,y)
    yyhat = lrnr.predict(XX)
    df = pd.DataFrame({
        'acc':[sklearn.metrics.accuracy_score(yy,yyhat)], 
        'pre':[sklearn.metrics.precision_score(yy,yyhat)], 
        'rec':[sklearn.metrics.recall_score(yy,yyhat)],
        'f1':[sklearn.metrics.f1_score(yy,yyhat)]}
    )    
    return df

def our_sampling1(df):
    cus_list = set(df.query('is_fraud==1').cc_num.tolist())
    return df.query("cc_num in @ cus_list")

In [4]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [5]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

## 시도

In [6]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [7]:
df50 = down_sample_textbook(df02)
df50.shape

In [8]:
12012*12012

In [9]:
df50 = df50.reset_index()

In [10]:
N = len(df50)

------------------------------------------------------------------------

### tr/test

In [11]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

In [12]:
df50_tr.is_fraud.mean().round(5), df50_test.is_fraud.mean().round(5)

In [13]:
df50_tr.shape, df50_test.shape

In [14]:
train_mask = np.concatenate((np.full(9009, True), np.full(3003, False)))
test_mask = np.concatenate((np.full(9009, False), np.full(3003, True)))
print("Train Mask:", train_mask)
print("Test Mask:", test_mask)

Train Mask: [ True  True  True ... False False False]
Test Mask: [False False False ...  True  True  True]

In [15]:
train_mask.shape, test_mask.shape

In [16]:
train_mask.sum(), test_mask.sum()

In [17]:
df50_com = pd.concat([df50_tr, df50_test])

In [18]:
df50_com = df50_com.reset_index()

------------------------------------------------------------------------

### aj_matrix

In [36]:
# edge_index_list2_com = []
# for i in range(N):
#     for j in range(N):
#         if df50_com['cc_num'][i] != df50_com['cc_num'][j]:  
#             edge = 0
#         else:
#             edge = 1
#         edge_index_list2_com.append([i, j, edge])

In [51]:
#np.save('edge_index_list2_50_com.npy', edge_index_list2_com)

edge_index_list2_com = np.load('edge_index_list2_50_com.npy')

In [19]:
# edge_index_list2_com

In [52]:
edge_index_list2_com.shape

In [53]:
num_nodes = 12012

In [54]:
aj_matrix = np.zeros((num_nodes, num_nodes))

In [55]:
aj_matrix

In [56]:
for i, j ,edge in edge_index_list2_com:
    aj_matrix[i][j] = edge

In [57]:
aj_matrix

In [37]:
# aj_matrix.shape

In [39]:
# np.save('aj_matrix.npy', aj_matrix)

In [41]:

# aj_matrix = np.load('aj_matrix.npy')

In [42]:
# aj_matrix

------------------------------------------------------------------------

------------------------------------------------------------------------

### weigt matirx

In [25]:
# edge_index_list = []
# for i in range(N):
#     for j in range(N):
#         time_difference = (df50_com['trans_date_trans_time'][i] - df50_com['trans_date_trans_time'][j]).total_seconds()
#         edge_index_list.append([i, j, time_difference])

In [41]:
# np.save('edge_index_list_50_com.npy', edge_index_list)

edge_index_list = np.load('edge_index_list_50_com.npy')

In [43]:
edge_index_list[:5]

In [44]:
edge_index = np.array(edge_index_list)
edge_index[:,2] = np.abs(edge_index[:,2])
theta = edge_index[:,2].mean()
theta

In [45]:
edge_index[:,2] = (np.exp(-edge_index[:,2]/theta)!=1) * np.exp(-edge_index[:,2]/theta)
edge_index

> w_matrix로 바꾸려고 하니까 형식이 \[i\]\[j\]가 맞지 않는다.!

In [92]:
# # 출력 형식 변경
# np.set_printoptions(formatter={'int': '{:d}'.format})

In [91]:
# # 원래 출력 형식으로 복원
# np.set_printoptions(formatter=None)


In [93]:
w_matrix = np.zeros((num_nodes, num_nodes))

In [94]:
w_matrix

In [97]:
for i, j ,time_difference in edge_index:
    i, j = int(i), int(j)
    w_matrix[i][j] = time_difference

In [98]:
w_matrix

In [99]:
w_matrix.shape

In [100]:
np.save('w_matrix.npy', w_matrix)

------------------------------------------------------------------------

In [19]:
# np.save('edge_index_list_plus.npy', edge_index_list_plus)

edge_index_list_plus = np.load('edge_index_list_plus.npy')

In [20]:
edge_index = np.array(edge_index_list_plus)

In [113]:
edge_index.shape

In [114]:
edge_index

In [21]:
edge_index[:,2] = np.abs(edge_index[:,2])
theta = edge_index[:,2].mean()
theta


In [22]:
edge_index[:,2] = (np.exp(-edge_index[:,2]/theta)!=1) * np.exp(-edge_index[:,2]/theta)
edge_index

In [23]:
edge_index_list_updated = edge_index.tolist()
mm = np.array(edge_index_list_updated)[:,2].mean()

In [24]:
selected_edges = [(int(row[0]), int(row[1])) for row in edge_index_list_updated if row[2] > mm]

In [25]:
edge_index_selected = torch.tensor(selected_edges, dtype=torch.long).t()

In [26]:
edge_index_selected.shape

In [27]:
np.save('edge_index_selected.npy', edge_index_selected)

------------------------------------------------------------------------

# pyg lesson6 따라하기

### data설정(x, edge_index, y)

In [28]:
x = torch.tensor([df50_com['amt']], dtype=torch.float).reshape(-1,1)

In [29]:
x

In [30]:
y = torch.tensor(df50_com['is_fraud'],dtype = torch.int64)

In [31]:
y

In [32]:
import torch_geometric

In [34]:
data = torch_geometric.data.Data(x=x, edge_index = edge_index_selected, y=y)

#train_mask = train_mask, test_mask = test_mask

In [35]:
data

------------------------------------------------------------------------

### GCNConv

In [37]:
gconv = torch_geometric.nn.GCNConv(1,4)
gconv

In [38]:
gconv(data.x, data.edge_index)

In [39]:
list(gconv.parameters())

In [40]:
_,W = list(gconv.parameters())
W

In [58]:
A = torch.tensor(aj_matrix, dtype=torch.float32)
Atilde = A + torch.eye(12012)

In [59]:
Atilde

> 엇? 자기자신은 = 0 을 .. 넣는게 여기든가!

In [60]:
Atilde@data.x@W.T/3, gconv(data.x,data.edge_index)

------------------------------------------------------------------------

> A를 선택하는거에 있어서 생각해보니 ,,,,,,, 잘못했다! 계속 그냥. .
> edge만 하는데 GConv에서는 weight를 통해서 edge를 골랐는데 .. 그럼
> 여기서도 그 mm값을 통해서 구해야할거같은데……이건 어떻게 계산하는 거지?